## Learning Sparse Reward Models

We consider the case of learning the reward function of the pendulum. In particular, we aim to learn the reward function $R:\mathcal{S}\times \mathcal{A}\rightarrow \mathbb{R}$, predicting the reward $\hat{r}_t$ from the current state $\mathbf{s}_t$ and action $\mathbf{a}_t$: $$\hat{r}_t=R(\mathbf{s}_t, \mathbf{a}_t).$$ We will approximate $R(\mathbf{s}_t, \mathbf{a}_t)$ using:
* a fully-connected neural network (fcnn_model),
* a fully-connected neural network sparsified by the L$_0$ regularization (sparsefcnn_model), and
* a SINDy-like architecture again sparsified by the L$_0$ regularization (l0sindy_model).

In [25]:
import gymnasium as gym
from utils.replay_buffer import ReplayBuffer
import torch
import numpy as np

# Set seeds
seed = 23524
torch.manual_seed(seed)
np.random.seed(seed)

We first create a training set composed of 1000 episodes of 200 steps each. The actions are sampled from a random policy.

In [ ]:
render = False
if render:
    env = gym.make('Pendulum-v1', g=9.81, render_mode="human")
else:
    env = gym.make('Pendulum-v1', g=9.81)
max_episodes = 1000
max_steps = 200

obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
buf_dim = int(max_episodes*max_steps)

# create training set
training_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=buf_dim)

for episode in range(max_episodes):
    observation, _ = env.reset()
    for steps in range(max_steps+1):
        action = env.action_space.sample() 
        next_observation, reward, terminated, truncated, _ = env.step(action)

        done = terminated or truncated
        training_buffer.store(observation, action, reward, next_observation, done)

        env.render()

        observation = next_observation

        if done:
            done = False
            break

print("Finished creating the training set")

Secondly, we create the test set that we will use to evaluate the models.

In [27]:
# create test set
max_episodes_test = 100
buf_dim = int(max_episodes*max_steps)

testing_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=buf_dim)

for episode in range(max_episodes_test):
    observation, _ = env.reset()
    for steps in range(max_steps + 1):
        action = env.action_space.sample()  
        next_observation, reward, terminated, truncated, _ = env.step(action)

        done = terminated or truncated
        testing_buffer.store(observation, action, reward, next_observation, done)

        env.render()

        observation = next_observation

        if done:
            done = False
            break

print("Finished creating the test set")

Finished creating the test set


After creating train and test set, we are now ready to train the three models. We utilize the same learning rate, batch size, and number of traning epochs.


In [28]:
# learning the reward function of the pendulum
from utils.models import FCNN, SparseFCNN, L0SINDy_reward
from utils.trainer import train_eval_reward_model
import torch

# number of hidden units used by the fcnn_model and the sparsefcnn_model
h_dim = 128

# shared hyperparameter of the experiment
lr = 3e-4
batch_size = 256
num_epochs = 500

Train and evaluate the fcnn_model.

In [ ]:
fcnn_model = FCNN(input_dim=obs_dim+act_dim, output_dim=1, h_dim=h_dim)

if torch.cuda.is_available():
    fcnn_model = fcnn_model.cuda()

optimizer_fcnn = torch.optim.Adam([
    {'params': fcnn_model.parameters()},
], lr=lr, weight_decay=0.0)

metrics_fcnn = train_eval_reward_model(fcnn_model, optimizer_fcnn, training_buffer, testing_buffer, batch_size, num_epochs)
print("Best testing error FCNN is {} and it was found at epoch {}".format(metrics_fcnn[2], metrics_fcnn[3]))

Train and evaluate the sparsefcnn_model.

In [ ]:
reg_coefficient = 0.0005
sparsefcnn_model = SparseFCNN(input_dim=obs_dim+act_dim, output_dim=1, h_dim=h_dim, lambda_coeff=reg_coefficient)

if torch.cuda.is_available():
    sparsefcnn_model = sparsefcnn_model.cuda()

optimizer_sparsefcnn = torch.optim.Adam([
    {'params': sparsefcnn_model.parameters()},
], lr=lr, weight_decay=0.0)

metrics_sparsefcnn = train_eval_reward_model(sparsefcnn_model, optimizer_sparsefcnn, training_buffer, testing_buffer,
                                               batch_size, num_epochs, l0=True)
print("Best testing error sparse FCNN is {} and it was found at epoch {}".format(metrics_sparsefcnn[2], metrics_sparsefcnn[3]))


Train and evaluate the l0sindy_model.

In [ ]:
# degree of the polynomial features
degree = 3

reg_coefficient = 0.0005
l0sindy_model = L0SINDy_reward(input_dim=obs_dim+act_dim, output_dim=1, degree=degree, lambda_coeff=reg_coefficient)

if torch.cuda.is_available():
    l0sindy_model = l0sindy_model.cuda()

optimizer_fcnn = torch.optim.Adam([
    {'params': l0sindy_model.parameters()},
], lr=lr, weight_decay=0.0)

metrics_l0sindy = train_eval_reward_model(l0sindy_model, optimizer_fcnn, training_buffer, testing_buffer, batch_size, num_epochs, l0=True)
print("Best testing error L0 SINDy is {} and it was found at epoch {}".format(metrics_l0sindy[2], metrics_l0sindy[3]))

l0sindy_model.print_equations()

Eventually, we plot the mean-squared error between the predictions and the ground-truth over the training and test set.

In [ ]:
# creating the plots
import matplotlib.pyplot as plt
import os

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Training and Evaluation Metrics')

data_train = {'FCNN': metrics_fcnn[0], 'SparseFCNN': metrics_sparsefcnn[0], 'L0SINDy': metrics_l0sindy[0]}
methods_train = list(data_train.keys())
values_train = list(data_train.values())

# creating the bar plot
ax1.bar(methods_train, values_train, color='maroon', width=0.4)

data_eval = {'FCNN': metrics_fcnn[2], 'SparseFCNN': metrics_sparsefcnn[2], 'L0SINDy': metrics_l0sindy[2]}
methods_eval = list(data_eval.keys())
values_eval = list(data_eval.values())

ax2.bar(methods_eval, values_eval, color='blue', width=0.4)

save_dir = "figures"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
fig.savefig('figures/LearningReward.png', dpi=300)